In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import logging as log
from enum import Enum
from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd
from dataclasses import dataclass, field
from typing import Union

import plotly.graph_objects as go

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Helper classes and functions

In [ ]:
#--------------------------------------------------
# Helper classes and enums
#--------------------------------------------------
@dataclass(frozen=False, order=False)
class CentralityAlgorithm:
    """
    A data-class to store centrality measures arguments and other options
    """
    name: str = field(default="")
    func: object = field(default=print)
    args: dict = field(default_factory=dict)
    options: dict = field(default_factory=dict)

@dataclass(frozen=False, order=False)
class CentralityAlgorithmOutput:
    """
    A data-class to store centrality measures results
    """
    name: str = field(default="")
    result: dict = field(default_factory=dict)
    options: dict = field(default_factory=dict)

class GRAPH_TYPE(str, Enum):
    BOTH = "both"
    DIRECTED = "directed"
    def __str__(self) -> str:
        return self.value

#--------------------------------------------------
# Helper functions
#--------------------------------------------------
def convert_to_normal(snake_case: str) -> str:
    """
    Converts a snake case string to a normal title string

    Input:  "in_degree_centrality"
    Output: "In Degree Centrality"
    """
    components = snake_case.split('_')
    return " ".join(x.title() for x in components)

def print_log(text: str):
    """
    Prints the log
    """
    print(f"[ log ]: {text}")
    # log.info(f"[ log ]: {text}")

def print_error(text: str):
    """
    Prints the error
    """
    print(f"[ error ]: {text}")
    # log.warn(f"[ error ]: {text}")

def undirected_to_directed(graph: nx.Graph) -> nx.DiGraph:
    """
    Converts an undirected graph to a directed graph
    """
    di_graph = nx.DiGraph()
    di_graph.add_edges_from(graph.edges())
    return di_graph

def get_graph(graph: nx.Graph, graph_type: str) -> Union[nx.Graph, nx.DiGraph]:
    """
    Returns the graph based on the graph type
    """
    if graph_type == str(GRAPH_TYPE.DIRECTED):
        return undirected_to_directed(graph)
    elif graph_type == str(GRAPH_TYPE.BOTH):
        return graph
    else:
        print_error(f"Unknown graph type: {graph_type}")
        return graph

def downsample_connected_graph(graph: nx.Graph, largest_cc: set, sample_size: int = 25, verbose: bool = False, plot: bool = False) -> nx.Graph:
    """
    Down-samples the graph to only contain "sample_size" number of nodes
    """
    # pick a random node from the largest connected component
    __node = np.random.choice(list(largest_cc))
    __total_nodes, __graph = 0, nx.Graph()
    
    for i, _ in enumerate(range(1, sample_size)):
        # get the ego graph of the node
        __ego_graph = nx.ego_graph(graph, __node, radius=i)
        __total_nodes = len(__ego_graph)
        if (__total_nodes >= sample_size) and not nx.is_empty(__graph): break
        if verbose: print_log(f"Node: '{__node}' yields '{__total_nodes}' ego-graph nodes after radius '{i}'")
        __graph = __ego_graph
    
    if plot: nx.draw(__graph, with_labels=True)
    return __graph

def make_graph_connected(graph: nx.Graph, sample_size: int = 25, sorted_cc: list = list(), verbose: bool = False, plot: bool = False) -> nx.Graph:
    """
    Connect the graph by adding more edges between the nodes of connected components
    """
    if sample_size <= 0: raise ValueError(f"Invalid sample size: {sample_size}")
    if len(sorted_cc) <= 1: raise ValueError(f"Invalid connected component list")
    
    largest_cc = sorted_cc[0]
    __first_node = list(largest_cc)[0]

    __graph = nx.subgraph(graph, largest_cc).copy()

    for i, cc in enumerate(sorted_cc[1:]):
        # return if adding another connected component will make the graph bigger than sample size
        if (len(__graph) + len(cc)) > sample_size:
            if verbose: print_log(f"Graph size: {len(__graph)} + CC({i + 1}): {len(cc)} > Sample Size: {sample_size}")
            break
        # add new edge to make the graph connected
        if verbose: print_log(f"Adding edge between '{__first_node}' and '{list(cc)[0]}'")
        __graph.add_edge(__first_node, list(cc)[0])

        # add the new connected component to the graph
        if verbose: print_log(f"Adding {len(cc)} nodes from CC({i + 1}) to the graph")
        __graph.add_edges_from(nx.subgraph(graph, cc).edges())

    if plot:
        fig, axes = plt.subplots(nrows=1, ncols=2)
        axes[0].set_title("Original Graph")
        nx.draw(graph, with_labels=True, ax=axes[0])
        axes[1].set_title("Final Graph")
        nx.draw(__graph, with_labels=True, ax=axes[1])
    return __graph

def sample_graph(graph: Union[nx.Graph, nx.DiGraph], sample_size: int = 25, verbose: bool = False, plot: bool = False) -> Union[nx.Graph, nx.DiGraph]:
    """
    Returns a sample graph of the given 'sample_size'
    """
    if sample_size <= 0: raise ValueError(f"Invalid sample size: {sample_size}")

    is_directed = nx.is_directed(graph) and isinstance(graph, nx.DiGraph)
    if is_directed: graph = graph.to_undirected()

    sorted_cc = sorted(nx.connected_components(graph), key=len, reverse=True)
    largest_cc = sorted_cc[0]

    __graph = nx.Graph()

    if sample_size > len(largest_cc):
        if verbose: print_error(f"Sample size {sample_size} is greater than the largest connected component size {len(largest_cc)}")
        if nx.is_connected(graph):
            if verbose: print_log(f"Graph is connected")
            __graph = graph.copy()
        else:
            # make graph bigger by connecting the connected components
            if verbose: print_log(f"Making graph connected")
            __graph = make_graph_connected(graph=graph, sample_size=sample_size, sorted_cc=sorted_cc, verbose=verbose, plot=plot)
    else:
        # make graph shorter by keeping 'sample_size' elements only and connecting them
        if verbose: print_log(f"Downsampling a larger graph")
        __graph = downsample_connected_graph(graph=graph, largest_cc=largest_cc, sample_size=sample_size, verbose=verbose, plot=plot)
    
    return undirected_to_directed(__graph) if is_directed else __graph

def compute_algorithms(graph: nx.Graph, algorithms: list, verbose: bool) -> list:
    """
    Computes the centrality measures for the given graph
    """
    centrality_measures = list()
    for i, algorithm in enumerate(algorithms):
        algorithm_name = convert_to_normal(algorithm.name)
        graph_direction = algorithm.options["direction"]

        if verbose:
            print_log(f"Running '{algorithm_name}' ({i+1}/{len(algorithms)})")
            print_log(f"Option: '{algorithm.options}'")

        # if graph is not directed or undirected, then skip -> invalid direction option
        if graph_direction != str(GRAPH_TYPE.BOTH) and graph_direction != str(GRAPH_TYPE.DIRECTED):
            if verbose: print_error(f"Invalid direction: '{graph_direction}'")
            continue

        __graph = get_graph(graph=graph, graph_type=graph_direction)

        try:
            __centrality = algorithm.func(__graph, **algorithm.args)
        except Exception as e:
            if verbose: print_error(f"Error while running '{algorithm_name}': {e}")
            continue

        # return the degree as the weight
        if isinstance(__centrality, list):
            __centrality = {node: __graph.degree(node) for node in __centrality}

        centrality_measures.append(CentralityAlgorithmOutput(algorithm.name, __centrality, algorithm.options))

        if verbose:
            print_log(f"Result: '{__centrality}'")
            print_log(f"="*50)
    return centrality_measures

In [ ]:
#import the data
#data source: https://www.inetbio.org/wormnet/downloadnetwork.php

parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, 'data')
data_file = join(data_dir, 'DM-HT.txt')
# data_file = join(data_dir, 'DM-HT_small.txt')

sample_size = 50

graph = nx.read_edgelist(data_file, nodetype=str, data=(("weight", float),))
graph = sample_graph(graph=graph, sample_size=sample_size, verbose=True, plot=True)
di_graph = undirected_to_directed(graph)

print(f"{graph}")

In [ ]:
edge_x = []
edge_y = []
for edge in graph.edges():
    x, y = edge
    x0, y0 = pos[x]
    x1, y1 = pos[y]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_y.append(y0)
    edge_y.append(y1)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
    )

In [ ]:
node_x = []
node_y = []
for node in graph.nodes():
    x, y = pos[node][0], pos[node][1]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

In [ ]:
node_degree_centrality = []
node_text = []

for node in graph_fly.nodes():
    node_degree_centrality.append(degree_centrality_graph_fly[node])
    node_text.append('# of degrees: '+ str(degree_centrality_graph_fly[node]))
node_trace.marker.color = node_degree_centrality
node_trace.text = node_text

In [ ]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:
nx.communicability_betweenness_centrality(graph_fly_sampleDraw_undirected)

In [ ]:
nx.in_degree_centrality(graph_fly_sampleDraw)

In [ ]:
nx.betweenness_centrality(graph_fly_sampleDraw)

In [ ]:
#nx.current_flow_betweenness_centrality(graph_fly_sampleDraw)
#for undirected graphs only

In [ ]:
nx.closeness_centrality(graph_fly_sampleDraw)

In [ ]:
# nx.eigenvector_centrality(graph_fly_sampleDraw)
# for connected graphs only

In [ ]:
#nx.current_flow_closeness_centrality(graph_fly_sampleDraw)
#for undirected graphs only

In [ ]:
#nx.communicability(graph_fly_sampleDraw)
#for undirected graphs only

In [ ]:
nx.load_centrality(graph_fly_sampleDraw)